In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from scipy.interpolate import interp1d
from datetime import datetime, timedelta
plt.rcParams["figure.dpi"] = 150

In [35]:
# 'MaizeGDD': {'Aer': 5.0,
#   'CCx': 0.96,  # Maximum canopy cover (fraction of soil cover)
#   'CDC': 0.01,     # Canopy decline coefficient (fraction per gdd/calendar day)   
#   'CDC_CD': 0.11691, 
#   'CGC': 0.012494,     #Canopy growth coefficient (fraction per gdd)
#   'CGC_CD': 0.16312,
#   'CalendarType': 2,
#   'CropType': 3,
#   'Determinant': 1.0,
#   'ETadj': 1.0,
#   'Emergence': 80.0,
#   'EmergenceCD': 6.0,
#   'Flowering': 180.0,
#   'FloweringCD': 13.0,
#   'GDD_lo': 0,
#   'GDD_up': 12.0,
#   'GDDmethod': 3,
#   'HI0': 0.48,
#   'HIstart': 880.0,
#   'HIstartCD': 66.0,
#   'Kcb': 1.05,
#   'Maturity': 1700.0,
#   'MaturityCD': 132.0,
#   'MaxRooting': 1409.0,
#   'MaxRootingCD': 108.0,


# crop.CGC = (
#                 np.log(
#                     (((0.98 * crop.CCx) - crop.CCx) * crop.CC0)
#                     / (-0.25 * (crop.CCx**2))
#                 )
#             ) / (-(crop.MaxCanopy - crop.Emergence))



#  tCD = crop.MaturityCD - crop.SenescenceCD
#             if tCD <= 0:
#                 tCD = 1

#             CCi = crop.CCx * (1 - 0.05 * (np.exp(((3.33 * crop.CDC_CD) / (crop.CCx + 2.29)) * tCD) - 1))
#             if CCi < 0:
#                 CCi = 0

#             tGDD = crop.Maturity - crop.Senescence
#             if tGDD <= 0:
#                 tGDD = 5

#             crop.CDC = ((crop.CCx + 2.29) * np.log((((CCi/crop.CCx) - 1) / -0.05) + 1)) / (3.33 * tGDD)


# print CDC and CGC params
#planting_date['canopy'] = 0.96/0.012494
# CGC = CCx/plnt_date2['canopy'].item()  

<!-- 
'MaizeGDD': {'Aer': 5.0,
  'CCx': 0.96,
  'CDC': 0.01,
  'CDC_CD': 0.11691,
  'CGC': 0.012494,
  'CGC_CD': 0.16312,
  'CalendarType': 2,
  'CropType': 3,
  'Determinant': 1.0,
  'ETadj': 1.0,
  'Emergence': 80.0,
  'EmergenceCD': 6.0,
  'Flowering': 180.0,
  'FloweringCD': 13.0,
  'GDD_lo': 0,
  'GDD_up': 12.0,
  'GDDmethod': 3,
  'HI0': 0.48,
  'HIstart': 880.0,
  'HIstartCD': 66.0,
  'Kcb': 1.05,
  'Maturity': 1700.0,
  'MaturityCD': 132.0,
  'MaxRooting': 1409.0,
  'MaxRootingCD': 108.0,
 -->

In [11]:
# 50p Planting/.. date functions

def interpolate_50P_dates(df, column="WC", text="e_date", g_stage = "Corn Planted" ):
    """
    Interpolate the DayOfYear for a given region (WC) and calculate corresponding harvesting dates.
    
    Parameters:
    - df: DataFrame containing columns 'Year', 'DayOfYear', and the specified 'column' (default: "WC")
    - column: the column name for soil water content (default: "WC")
    - text: the label for the resulting date column (default: "h_date")
  
    
    Returns:
    - interpolated_df: DataFrame with years and interpolated harvesting dates
    """
#     filtering df
    df = df[["G.Stage",column, "time", "Year","DayOfYear" ]]
    df = df [df["G.Stage"] ==g_stage]
    df[column] = df[column].astype(float)
    

    # Create a dictionary to store interpolated values
    desired_p=50  #  - desired_p: the desired percentage for interpolation (default: 50)
    interpolated_values = {'Year': [], f'Interpolated_DayOfYear for {text}': []}
    
    # Iterate over unique years
    for year in df['Year'].unique():
        year_data = df[df['Year'] == year]
        
        # Sort the data by the specified column for interpolation
        year_data = year_data.sort_values(by=column)
        
        # Check if desired SW is within the range of the data
        if (desired_p >= year_data[column].min()) and (desired_p <= year_data[column].max()):
            # Perform linear interpolation
            f = interp1d(year_data[column], year_data['DayOfYear'])
            interpolated_day = f(desired_p)
            
            interpolated_values['Year'].append(year)
            interpolated_values[f'Interpolated_DayOfYear for {text}'].append(np.round(interpolated_day))
    
    # Create a new DataFrame from the interpolated values
        interpolated_df = pd.DataFrame(interpolated_values)
        interpolated_df[text] = [datetime(year, 1, 1) + timedelta(int(day) - 1) for year, day in zip(interpolated_df['Year'], interpolated_df[f'Interpolated_DayOfYear for {text}'])]
    
    return interpolated_df



def interp_date_df(df, column = "WC"):
    
    pdatewc = interpolate_50P_dates(df=df, column=column, text="p_date", g_stage = "Corn Planted" )
    emdatewc = interpolate_50P_dates(df=df, column=column, text="e_date", g_stage = "Corn Emerged" )
    mdatewc = interpolate_50P_dates(df=df, column=column, text="m_date", g_stage = "Corn Matured" )
    hdatewc = interpolate_50P_dates(df=df, column=column, text="h_date", g_stage = "Corn Harvested" )
    wc_interpol = pd.concat([pdatewc,emdatewc, mdatewc,hdatewc  ], axis =1)
    wc_interpol.drop(wc_interpol.Year, axis =1, inplace =True)
    wc_interpol["Year"] = wc_interpol["p_date"].dt.year
    wc_interpol["FieldID"] = column
    return wc_interpol
    


In [12]:


def GDD_calc(date_df, wdf, g_s__date="m_date", text="M_GDD_SW", p_date=None, method=3):
    """
    Calculate Growing Degree Days (GDD) for a given range of years using specified method.

    Parameters:
    - date_df: DataFrame containing growing stage dates.
    - wdf: DataFrame containing weather data.
    - g_s__date: Column name for the harvesting date in date_df (default is "m_date").
    - text: Column name for GDD calculation result in the output DataFrame (default is "M_GDD_SW").
    - p_date: Column name for the planting date in date_df (default is None, which sets it to "p_date").
    - method: Integer (1, 2, or 3) to select the GDD calculation method (default is 1).
    
    Returns:
    - results_df: DataFrame containing the total GDD for each year.
    """
    
    if p_date is None:
        p_date = "p_date"
        
    Tbase = 8  # Base temperature in °C
    Tupper = 30  # Upper temperature in °C

    # Create a DataFrame to store results
    results_df = pd.DataFrame(columns=['Year', text])

    # Loop through each year
    for year in range(2011, 2023):
        # Fetch planting and harvesting dates for the current year from your 'DataFrame'
        planting_date = pd.to_datetime(date_df.loc[date_df['Year'] == year, p_date].values[0])
        harvesting_date = pd.to_datetime(date_df.loc[date_df['Year'] == year, g_s__date].values[0])

        # Filter the weather DataFrame for the current year
        wdf_year = wdf[wdf.index.year == year].copy()

        if method == 1:
            # Method 1: Calculate GDD using average temperature directly
            def calculate_aquacrop_gdd(df):
                average_temperature = (df['MaxTemp'] + df['MinTemp']) / 2

                # Apply conditions for Tavg
                if average_temperature < Tbase:
                    tavg = Tbase
                elif average_temperature > Tupper:
                    tavg = Tupper
                else:
                    tavg = average_temperature

                gdd = max(0, tavg - Tbase)  # if the average_temperature is below the base temperature, GDD is set to zero
                return gdd if planting_date <= df['Date'] <= harvesting_date else 0

        elif method == 2:
            # Method 2: Adjust Tx and Tn before calculating average temperature
            def calculate_aquacrop_gdd(df):
                Tx = df['MaxTemp']
                Tn = df['MinTemp']

                # Adjust Tx
                if Tx > Tupper:
                    Tx_star = Tupper
                elif Tx < Tbase:
                    Tx_star = Tbase
                else:
                    Tx_star = Tx

                # Adjust Tn
                if Tn > Tupper:
                    Tn_star = Tupper
                elif Tn < Tbase:
                    Tn_star = Tbase
                else:
                    Tn_star = Tn

                # Calculate average temperature using adjusted values
                tavg = (Tx_star + Tn_star) / 2

                # Calculate GDD
                gdd = max(0, tavg - Tbase)  # if the average temperature is below the base temperature, GDD is set to zero
                return gdd if planting_date <= df['Date'] <= harvesting_date else 0

        elif method == 3:
            # Method 3: Adjust only Tx before calculating average temperature
            def calculate_aquacrop_gdd(df):
                Tx = df['MaxTemp']
                Tn = df['MinTemp']

                # Adjust Tx
                if Tx > Tupper:
                    Tx_star = Tupper
                elif Tx < Tbase:
                    Tx_star = Tbase
                else:
                    Tx_star = Tx

                # If Tn exceeds Tupper, set Tn to Tupper
                if Tn > Tupper:
                    Tn = Tupper

                # Calculate average temperature using adjusted Tx
                tavg = (Tx_star + Tn) / 2

                # Apply base temperature condition to tavg
                if tavg < Tbase:
                    tavg = Tbase

                # Calculate GDD
                gdd = max(0, tavg - Tbase)  # if the average temperature is below the base temperature, GDD is set to zero
                return gdd if planting_date <= df['Date'] <= harvesting_date else 0

        # Apply the GDD calculation
        wdf_year[text] = wdf_year.apply(calculate_aquacrop_gdd, axis=1)

        # Calculate the total GDD accumulation for the current year
        total_gdd = wdf_year[text].sum()

        # Store the results in the results DataFrame
        results_df = results_df.append({'Year': year, text: total_gdd}, ignore_index=True)

    return results_df

def GDD_df(df, wdf, fid="WC", method= 3):
    """
    Calculate GDD for emergence and maturity stages and return a summary DataFrame.

    Parameters:
    - df: DataFrame containing growing stage dates.
    - wdf: DataFrame containing weather data.
    - fid: Field ID (default is "WC").

    Returns:
    - GDD: DataFrame containing mean GDD for emergence and maturity stages, with the Field ID.
    """
    
    emerg = GDD_calc(df, wdf, g_s__date="e_date", text="e_GDD", method=method)
    maturity = GDD_calc(df, wdf, g_s__date="m_date", text="m_GDD", method=method)
    GDD = pd.DataFrame({"Emergence": round(emerg.mean()[1]), "Maturity": round(maturity.mean()[1]), "FieldID": fid}, index=[0])

    return GDD


need 4 dates (planting, emergence,maturity,and harvesting), 2 gdd (emergence, maturity)

# SW_cal

In [35]:
# sw weather df

sw = pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/SW_fields.csv")
sw= sw[['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET', 'Date']]
sw.Date = pd.to_datetime(sw.Date)
wdf =sw
wdf['Date'] = pd.to_datetime(wdf['Date'], format='%y-%m-%d')

wdf.index = wdf['Date']
wdf

,MinTemp,MaxTemp,Precipitation,ReferenceET,Date
Date,,,,,
2006-01-01,-4.267836,12.625818,0.0,1.572434,2006-01-01
2006-01-02,-3.675269,15.338580,0.0,2.176727,2006-01-02
2006-01-03,-1.463397,21.520454,0.0,4.418460,2006-01-03
2006-01-04,-3.645713,13.656253,0.0,1.820799,2006-01-04
2006-01-05,-3.958295,11.505593,0.0,2.511666,2006-01-05
...,...,...,...,...,...
2023-12-26,-2.435348,1.124070,0.1,1.166667,2023-12-26
2023-12-27,-1.033625,3.563650,0.0,1.066667,2023-12-27
2023-12-28,-6.265374,1.107535,0.0,0.500000,2023-12-28


In [36]:
# planting date SW fields

ffty_p_interp_date= pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/"+
          "old_codes_And_new_mac_codes/Python_docs/Data/fifty_p_interp_date.csv")
ffty_p_interp_date

,Unnamed: 0,Interpolated_DayOfYear for 50p corn planting,p_date,Interpolated_DayOfYear for 50p corn Emerg,e_date,Interpolated_DayOfYear for 50p corn silk,s_date,Interpolated_DayOfYear for 50p corn mat,m_date,Interpolated_DayOfYear for 50p corn Harvesting,h_date,Year
0,0,130.0,2022-05-10,142.416667,2022-05-22,205.774194,2022-07-24,259.900000,2022-09-16,280.400000,2022-10-07,2022
1,1,130.0,2021-05-10,144.000000,2021-05-24,197.351351,2021-07-16,259.769231,2021-09-16,275.000000,2021-10-02,2021
2,2,129.0,2020-05-08,142.636364,2020-05-21,198.166667,2020-07-16,261.684211,2020-09-17,280.230769,2020-10-06,2020
3,3,135.0,2019-05-15,149.636364,2019-05-29,203.875000,2019-07-22,258.263158,2019-09-15,282.800000,2019-10-09,2019
4,4,130.0,2018-05-10,140.820513,2018-05-20,197.636364,2018-07-16,261.037037,2018-09-18,282.000000,2018-10-09,2018
5,5,136.0,2017-05-16,146.800000,2017-05-26,199.037037,2017-07-18,258.307692,2017-09-15,290.400000,2017-10-17,2017
6,6,131.0,2016-05-10,144.842105,2016-05-23,197.962963,2016-07-15,259.407407,2016-09-15,278.130435,2016-10-04,2016
7,7,132.0,2015-05-12,147.500000,2015-05-27,199.804878,2015-07-18,257.636364,2015-09-14,276.090909,2015-10-03,2015
8,8,129.0,2014-05-09,143.000000,2014-05-23,198.800000,2014-07-17,264.631579,2014-09-21,288.644444,2014-10-15,2014
9,9,136.0,2013-05-16,148.214286,2013-05-28,203.205882,2013-07-22,266.318182,2013-09-23,282.545455,2013-10-09,2013


In [15]:
ffty_p_interp_date.mean()

Unnamed: 0                                           5.500000
Interpolated_DayOfYear for 50p corn planting       130.666667
Interpolated_DayOfYear for 50p corn Emerg          144.254121
Interpolated_DayOfYear for 50p corn silk           199.943351
Interpolated_DayOfYear for 50p corn mat            259.166528
Interpolated_DayOfYear for 50p corn Harvesting     279.869021
Year                                              2016.500000
dtype: float64

## SW_GDD calculation

In [37]:


sw_GDD =GDD_df (ffty_p_interp_date,wdf, "SW", method =3)
sw_GDD

,Emergence,Maturity,FieldID
0,121,1815,SW


## 50 Percent date interpolation

In [38]:
import pandas as pd
df = pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/old_codes_And_new_mac_codes/Python_docs/Data/2022_2010_raw_crop_growthdata.csv")
idx =df[df["NW"]=='Percent'].index
# idx
df = df.drop(idx, axis =0)
df['time'] = pd.to_datetime(df['Date'], format='%B %d, %Y')
df['t2'] = pd.to_datetime(df['Date'], format='%B %d, %Y')
# df.index = df['time'] 
df =df.drop(["Unnamed: 0"], axis =1)
dfc= df[df["G.Stage"].str.contains("Corn",na=False)].sort_values("time", ascending = False)
dfc["G.Stage"] = dfc["G.Stage"].replace(['Corn Dent', 'Corn Mature','Corn Silking' ], ['Corn Dented','Corn Matured','Corn Silked' ])
dfc['DayOfYear'] = dfc['time'].dt.dayofyear
dfc['Year'] = dfc['time'].dt.year
dfc.reset_index()

dfc

,G.Stage,NW,WC,SW,NC,C SC,NE,EC,SE,STATE,C,SC,Date,Filename,SC NE,time,t2,DayOfYear,Year
80,Corn Harvested,89,96.0,97.0,95,NaN,99.0,98.0,100.0,96.0,96.0,100.0,"November 14, 2022",2022_KS-weekly1114.pdf,NaN,2022-11-14,2022-11-14,318,2022
111,Corn Harvested,85,94.0,95.0,92,NaN,97.0,96.0,100.0,93.0,94.0,98.0,"November 7, 2022",2022_KS-weekly1107.pdf,NaN,2022-11-07,2022-11-07,311,2022
104,Corn Harvested,82,89.0,91.0,85,NaN,95.0,95.0,99.0,90.0,92.0,97.0,"October 31, 2022",2022_KS-weekly1031.pdf,NaN,2022-10-31,2022-10-31,304,2022
50,Corn Harvested,72,74.0,85.0,78,NaN,92.0,90.0,97.0,83.0,90.0,93.0,"October 24, 2022",2022_KS-weekly1024.pdf,NaN,2022-10-24,2022-10-24,297,2022
43,Corn Harvested,51,62.0,75.0,66,NaN,89.0,83.0,96.0,73.0,88.0,87.0,"October 17, 2022",2022_KS-weekly1017.pdf,NaN,2022-10-17,2022-10-17,290,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4881,Corn Planted,0,0.0,2.0,1.0,NaN,NaN,8.0,16.0,5.0,9,NaN,"April 11, 2011",2011_wkly0410.pdf,11 5,2011-04-11,2011-04-11,101,2011
5166,Corn Harvested,93.0,96.0,100.0,93.0,NaN,NaN,98.0,100.0,97.0,100.0,NaN,"November 1, 2010",2010_wkly1031.pdf,100 97,2010-11-01,2010-11-01,305,2010
5185,Corn Harvested,81.0,93.0,98.0,87.0,NaN,NaN,96.0,99.0,93.0,99.0,NaN,"October 25, 2010",2010_wkly1024.pdf,99 94,2010-10-25,2010-10-25,298,2010
5145,Corn Harvested,77.0,89.0,95.0,78.0,NaN,NaN,92.0,99.0,89.0,98.0,NaN,"October 18, 2010",2010_wkly1017.pdf,99 88,2010-10-18,2010-10-18,291,2010


In [39]:
wc_interpol= interp_date_df(dfc, column = "WC")
wc_interpol

,Interpolated_DayOfYear for p_date,p_date,Interpolated_DayOfYear for e_date,e_date,Interpolated_DayOfYear for m_date,m_date,Interpolated_DayOfYear for h_date,h_date,Year,FieldID
0,141.0,2022-05-21,153.0,2022-06-02,270.0,2022-09-27,287.0,2022-10-14,2022,WC
1,142.0,2021-05-22,156.0,2021-06-05,261.0,2021-09-18,278.0,2021-10-05,2021,WC
2,137.0,2020-05-16,153.0,2020-06-01,260.0,2020-09-16,285.0,2020-10-11,2020,WC
3,144.0,2019-05-24,156.0,2019-06-05,267.0,2019-09-24,285.0,2019-10-12,2019,WC
4,136.0,2018-05-16,145.0,2018-05-25,266.0,2018-09-23,296.0,2018-10-23,2018,WC
5,147.0,2017-05-27,150.0,2017-05-30,271.0,2017-09-28,298.0,2017-10-25,2017,WC
6,135.0,2016-05-14,152.0,2016-05-31,266.0,2016-09-22,284.0,2016-10-10,2016,WC
7,134.0,2015-05-14,146.0,2015-05-26,259.0,2015-09-16,276.0,2015-10-03,2015,WC
8,135.0,2014-05-15,150.0,2014-05-30,271.0,2014-09-28,290.0,2014-10-17,2014,WC
9,140.0,2013-05-20,153.0,2013-06-02,269.0,2013-09-26,284.0,2013-10-11,2013,WC


In [19]:
wc_interpol.mean()

Interpolated_DayOfYear for p_date     137.666667
Interpolated_DayOfYear for e_date     150.250000
Interpolated_DayOfYear for m_date     264.250000
Interpolated_DayOfYear for h_date     284.916667
Year                                 2016.500000
dtype: float64

## WC GDD

In [20]:
# weather data

wc = pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/WC_fields.csv")
wc= wc[['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET', 'Date']]
wc.Date = pd.to_datetime(wc.Date)
wc['Date'] = pd.to_datetime(wc['Date'], format='%y-%m-%d')
wc.index = wc['Date']

In [21]:
wc_GDD= GDD_df (wc_interpol,wc, fid ="WC" )   
wc_GDD

,Emergence,Maturity,FieldID
0,132,1804,WC


## NC_dates

In [22]:
nc_interpol= interp_date_df(dfc, column = "NC")
nc_interpol

,Interpolated_DayOfYear for p_date,p_date,Interpolated_DayOfYear for e_date,e_date,Interpolated_DayOfYear for m_date,m_date,Interpolated_DayOfYear for h_date,h_date,Year,FieldID
0,137.0,2022-05-17,155.0,2022-06-04,257.0,2022-09-14,281.0,2022-10-08,2022,NC
1,128.0,2021-05-08,143.0,2021-05-23,259.0,2021-09-16,287.0,2021-10-14,2021,NC
2,128.0,2020-05-07,143.0,2020-05-22,257.0,2020-09-13,285.0,2020-10-11,2020,NC
3,139.0,2019-05-19,154.0,2019-06-03,265.0,2019-09-22,298.0,2019-10-25,2019,NC
4,131.0,2018-05-11,142.0,2018-05-22,260.0,2018-09-17,299.0,2018-10-26,2018,NC
5,126.0,2017-05-06,146.0,2017-05-26,257.0,2017-09-14,295.0,2017-10-22,2017,NC
6,122.0,2016-05-01,133.0,2016-05-12,263.0,2016-09-19,285.0,2016-10-11,2016,NC
7,126.0,2015-05-06,136.0,2015-05-16,259.0,2015-09-16,278.0,2015-10-05,2015,NC
8,125.0,2014-05-05,143.0,2014-05-23,263.0,2014-09-20,292.0,2014-10-19,2014,NC
9,136.0,2013-05-16,148.0,2013-05-28,270.0,2013-09-27,292.0,2013-10-19,2013,NC


In [23]:
nc_interpol.mean()

Interpolated_DayOfYear for p_date     128.500000
Interpolated_DayOfYear for e_date     142.416667
Interpolated_DayOfYear for m_date     259.416667
Interpolated_DayOfYear for h_date     286.833333
Year                                 2016.500000
dtype: float64

## NC_GDD

In [24]:
# NC_weather data
nc = pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/NC_fields.csv")
nc = nc [['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET', 'Date']]
nc.Date = pd.to_datetime(nc .Date)
nc['Date'] = pd.to_datetime(nc['Date'], format='%y-%m-%d')
nc.index = nc['Date']


In [26]:
nc_GDD= GDD_df (nc_interpol,nc, fid ="NC" )   
nc_GDD

,Emergence,Maturity,FieldID
0,109,1723,NC


# NW

In [27]:
# dates
nw_interpol= interp_date_df(dfc, column = "NW")
nw_interpol

,Interpolated_DayOfYear for p_date,p_date,Interpolated_DayOfYear for e_date,e_date,Interpolated_DayOfYear for m_date,m_date,Interpolated_DayOfYear for h_date,h_date,Year,FieldID
0,140.0,2022-05-20,156.0,2022-06-05,258.0,2022-09-15,289.0,2022-10-16,2022,NW
1,144.0,2021-05-24,157.0,2021-06-06,264.0,2021-09-21,284.0,2021-10-11,2021,NW
2,138.0,2020-05-17,151.0,2020-05-30,264.0,2020-09-20,289.0,2020-10-15,2020,NW
3,146.0,2019-05-26,159.0,2019-06-08,270.0,2019-09-27,300.0,2019-10-27,2019,NW
4,140.0,2018-05-20,147.0,2018-05-27,261.0,2018-09-18,300.0,2018-10-27,2018,NW
5,152.0,2017-06-01,161.0,2017-06-10,269.0,2017-09-26,301.0,2017-10-28,2017,NW
6,136.0,2016-05-15,154.0,2016-06-02,264.0,2016-09-20,289.0,2016-10-15,2016,NW
7,135.0,2015-05-15,155.0,2015-06-04,265.0,2015-09-22,283.0,2015-10-10,2015,NW
8,132.0,2014-05-12,148.0,2014-05-28,271.0,2014-09-28,296.0,2014-10-23,2014,NW
9,137.0,2013-05-17,148.0,2013-05-28,265.0,2013-09-22,284.0,2013-10-11,2013,NW


In [28]:
# weather data
nw= pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/NW_fields.csv")

nw = nw [['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET', 'Date']]
nw.Date = pd.to_datetime(nw.Date)
nw['Date'] = pd.to_datetime(nw['Date'], format='%y-%m-%d')
nw.index = nw['Date']

In [29]:
nw_GDD= GDD_df (nw_interpol,nw, fid ="NW" )   
nw_GDD

,Emergence,Maturity,FieldID
0,127,1699,NW


In [31]:
# method#3
all_GDD = pd.concat([sw_GDD, wc_GDD, nc_GDD, nw_GDD])
all_GDD

,Emergence,Maturity,FieldID
0,121,1815,SW
0,132,1804,WC
0,109,1723,NC
0,127,1699,NW


In [74]:
method#2
all_GDD = pd.concat([sw_GDD, wc_GDD, nc_GDD, nw_GDD])
all_GDD

,Emergence,Maturity,FieldID
0,121,1815,SW
0,135,1810,WC
0,121,1741,NC
0,134,1711,NW


In [248]:
# method #1

all_GDD = pd.concat([sw_GDD, wc_GDD, nc_GDD, nw_GDD])
all_GDD

,Emergence,Maturity,FieldID
0,125,2013,SW
0,136,2003,WC
0,112,1895,NC
0,130,1876,NW


In [32]:
# all_GDD.to_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/all_fields_GDD.csv")


In [33]:
gdd_df =pd.read_csv("/Users/m089r172/Library/CloudStorage/OneDrive-UniversityofKansas/excel files/all_fields_GDD.csv")

gdd_df

,Unnamed: 0,Emergence,Maturity,FieldID
0,0,121,1815,SW
1,0,132,1804,WC
2,0,109,1723,NC
3,0,127,1699,NW


125